In [1]:
import cv2
import imutils
import numpy as np
import cupy as cp
import random
from os import listdir
import io
from IPython.display import display
from PIL import Image
import time
import cv2

from tensorflow.keras.preprocessing.image import ImageDataGenerator

%run ./variables.ipynb
%run ./utils.ipynb

In [9]:
def list_images(data_path, diatoms=True):
    # List all images in dataset and putting them in a dict {"key": "imgpath"}
    images_path = []
    for path in data_path: images_path.extend([(path, f) for f in listdir(path) if isfile(join(path, f))])
    images_dict = {}
    selected_taxons = get_selected_taxons()
    for image_path in images_path:
        if len(image_path[1].split('.'))>=2 and image_path[1].split('.')[-1]=='png': 
            # defining key
            if diatoms: 
                key = image_path[1].split('_')[1] #key is taxon
                if not key in selected_taxons:
                    key = None
            else: 
                key="mono"
            if not key is None:
                images_dict.setdefault(key,[]).append(os.path.join(image_path[0],image_path[1]))
    return images_dict   

In [ ]:
images_dict = list_images(DATASET_PATH, diatoms=True)
start_time = time.time()
total_n_taxons = len(images_dict.keys())
a = display(str(total_n_taxons)+" taxons in dataset! Output in: "+SAVE_PATH,display_id=True)
b = display("Progress: 0/"+str(total_n_taxons),display_id=True)
for n, taxon in enumerate(images_dict):
    images_paths = images_dict[taxon]
    taxon_images = []
    for path in images_paths:
        raw_image = np.array(Image.open(path))
        if len(raw_image.shape)==3: raw_image = cv2.cvtColor(raw_image, cv2.COLOR_BGR2GRAY)
        squared_image = convert_to_square(raw_image)
        squared_image = cv2.resize(squared_image, dsize=(256, 256), interpolation=cv2.INTER_CUBIC)
        squared_image = expand(squared_image)
        taxon_images.append(squared_image)
    taxon_images = np.array(taxon_images)
    n_images = taxon_images.shape[0]
    
    datagen = ImageDataGenerator(rotation_range=90, 
                             brightness_range=[0.8,1.2], 
                             horizontal_flip=True, 
                             vertical_flip=True,
                             fill_mode='nearest') 
    path = os.path.join(SAVE_PATH, taxon+"/")
    datagen.fit(taxon_images)
    check_dirs(path)
    i = 0
    for batch in datagen.flow(taxon_images, batch_size=1, save_to_dir=path, save_prefix=taxon, save_format='png'):
        i+=1
        if i>=10*n_images:
            break
    b.update("Progress: "+str(n+1)+"/"+str(total_n_taxons)+" ("+taxon+" pending...)")
print("Job finished in ", (time.time()-start_time), "s!")

'185 taxons in dataset! Output in: /mnt/nvme-storage/pfauregi/datasets/augmented/'

'Progress: 55/185 (NSOC pending...)'